In [1]:
"""
There are two variants to try: in one h and m encoding is first converted to noise...


"""
%run common/imp_and_args1.ipynb
if ntbk:
    sys.argv = ['main',
                '--niter', '10000', "--loadPreGPath", "preG.mdl" ]#, ]#, "--critic_iters", "5"] , '--useLN'
print(sys.argv)
#Now add some parser argument here, commented examples follow
#Boolean, action is what it will be set to when passed
#parser.add_argument('--notLog', action='store_true', help='if passed, no logging will happen. Logging is off for jupyter anyways.;')
#String
#parser.add_argument('--opFolder', default='', help='Override the logging folder.')
#int
#parser.add_argument('--seed', type=int, default=1, help='Seed for pyTorcj.')
#and float
#parser.add_argument('--clamp_lower', type=float, default=-0.01)


parser.add_argument('--loadPreGPath', default='', 
                    help="if passed, PreG would be loaded from this path")


parser.add_argument('--newGetZ', action='store_true', 
                    help="if passed, GetZ won't be created form D, "+
                    " but will be created from scratch. And Batch "+
                    " Normalization would be used.")

parser.add_argument('--useLN', action='store_true', 
                    help="if passed, Layer Norm would be used, else batch norm.")
parser.add_argument('--getZ', action='store_true', 
                    help="if passed, then inverse mapping will get z, else it will get L.")

%run common/args_and_rest.ipynb
g_nz = opt.hm_gen_size + opt.nz
fixed_noise = torch.randn(opt.batchSize, g_nz)#, 1, 1)
if cuda:
    fixed_noise = fixed_noise.cuda(gpu)
%run common/Utils.ipynb
%config Completer.use_jedi = False

['main', '--niter', '10000', '--loadPreGPath', 'preG.mdl']
Namespace(adam=False, batchSize=64, beta1=0.5, beta2=0.9, clamp_lower=-0.01, clamp_upper=0.01, critic_iters=5, dataroot='/home/aknirala/data/lsun', dataset='diverse_clocks', dim=64, fineSize=64, getZ=False, hm_gen_size=30, lambda_value=10.0, loadPreGPath='preG.mdl', loadSize=96, lrD=0.0002, lrG=0.0002, nEpochs=5, nc=3, ndf=64, newGetZ=False, ngf=64, ngpu=1, niter=10000, noCuda=False, noise='normal', notLog=False, nz=100, opFolder='', resumeFldr='', seed=1, useLN=False, workers=8)
OP folder won't be created as we are in jupyter notebook and logging is off
Arguments are:  Namespace(adam=False, batchSize=64, beta1=0.5, beta2=0.9, clamp_lower=-0.01, clamp_upper=0.01, critic_iters=5, dataroot='/home/aknirala/data/lsun', dataset='diverse_clocks', dim=64, fineSize=64, getZ=False, hm_gen_size=30, lambda_value=10.0, loadPreGPath='preG.mdl', loadSize=96, lrD=0.0002, lrG=0.0002, nEpochs=5, nc=3, ndf=64, newGetZ=False, ngf=64, ngpu=1, nite

In [ ]:
dset2 = same_style_clocks(randSeed=1, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize(64),
        transforms.CenterCrop(64),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]))
dload2 = torch.utils.data.DataLoader(dset2, batch_size=opt.batchSize,
                                            shuffle=True, num_workers=int(opt.workers))
same_style_iter = iter(dload2)
data = same_style_iter.next()
clocks = data[0]#.reshape(-1, 3, 64, 64)
plt.imshow(vutils.make_grid(
     clocks[:16],
    padding = 2, nrow = 4, normalize = True
).numpy().transpose([1, 2, 0]))


In [ ]:
%run freezed/WGAN_PG.ipynb

In [ ]:
class PreGen(nn.Module):
    def __init__(self):
        super(PreGen, self).__init__()
        global opt
        self.main = nn.Sequential()
        L1 = nn.Linear(72, 1024)
        relu1 = nn.ReLU()
        L2 = nn.Linear(1024, 1024)
        relu2 = nn.ReLU()
        L3 = nn.Linear(1024, opt.hm_gen_size)
        self.main.add_module("PreGen_L1", L1)
        self.main.add_module("PreGen_relu1", relu1)
        self.main.add_module("PreGen_L2", L2)
        self.main.add_module("PreGen_relu2", relu2)
        self.main.add_module("PreGen_L3", L3)
    def forward(self, x):
        return self.main(x)

In [ ]:
class PreDis(nn.Module):
    def __init__(self):
        super(PreDis, self).__init__()
        global opt
        self.main = nn.Sequential()
        L1 = nn.Linear(opt.hm_gen_size, 1024)
        relu1 = nn.ReLU()
        L2 = nn.Linear(1024, 1024)
        relu2 = nn.ReLU()
        L3 = nn.Linear(1024, 1)
        self.main.add_module("PreGen_L1", L1)
        self.main.add_module("PreGen_relu1", relu1)
        self.main.add_module("PreGen_L2", L2)
        self.main.add_module("PreGen_relu2", relu2)
        self.main.add_module("PreGen_L3", L3)
    def forward(self, x):
        return self.main(x)

In [ ]:
#We would use WGAN_PG to train this
preG = PreGen()
optPreG = optim.Adam(preG.parameters(), lr=opt.lrD, betas=(opt.beta1, opt.beta2))

one = torch.FloatTensor([1])
mone = one * -1
if cuda:
    one  = one.cuda(gpu)
    mone = mone.cuda(gpu)
    preG = preG.cuda(gpu)
    

In [ ]:
def getRandomHM():
    global opt
    H = []
    M = []
    h_eye = torch.eye(12)
    m_eye = torch.eye(60)
    for _ in range(opt.batchSize):
        H.append(np.random.randint(12))
        M.append(np.random.randint(60))
    H = h_eye[H].clone().cuda(gpu)
    M = m_eye[M].clone().cuda(gpu)
    hm = torch.cat([H, M], axis=1).cuda(gpu)
    return hm

In [ ]:
if opt.loadPreGPath != '':
    preG_chkpt = torch.load(opt.loadPreGPath)
    preG.load_state_dict(preG_chkpt)
    preG = preG.cuda()
    preG.eval()
else:
    D_real = 1
    D_fake = -1
    gradient_penalty = opt.lambda_value
    G_cost = 1
    iters = -1
    R_COST = []
    F_COST = []
    W_DIST = []
    D_COST = []
    G_COST = []
    GP = []
    preD = PreDis().cuda(gpu)
    optPreD = optim.Adam(preD.parameters(), lr=opt.lrD, betas=(opt.beta1, opt.beta2))

    for epoch in range(2):
        plog("Epoch: ", epoch, " of ", opt.nEpochs, " iters: ", iters)
        i = 0
        pbar = tqdm(range(10000))
        for i in pbar:
            pbar.set_description( (" D_real {:.3f}, D_fake {:.3f}, "\
                +"gradient_penalty {:.3f}, G_cost {:.3f},  {}").format(
                float(D_real), float(D_fake), 
                float(gradient_penalty), float(G_cost), iters))
            iters += 1
            ############################
            # (1) Update D network
            ###########################
            for p in preD.parameters():  # reset requires_grad
                p.requires_grad = True  # they are set to False below in netG update
            #
            for iter_d in range(opt.critic_iters):
                optPreD.zero_grad()
                # train with real ####################
                data = torch.randn(opt.batchSize, opt.hm_gen_size).cuda(gpu)
                D_real = preD(data.float())
                D_real = D_real.mean().reshape(1)
                if iters == 0 and iter_d == 0: plog("D_real.shape: ",D_real.shape)
                D_real.backward(one)
                #
                # train with fakes ##########################
                hm = getRandomHM()
                with torch.no_grad():
                    fakes = preG(hm).data
                D_fake = preD(fakes)
                D_fake = D_fake.mean().reshape(1)
                if iters == 0 and iter_d == 0: plog("D_fake.shape: ",D_fake.shape)
                D_fake.backward(mone)
                #
                # train with gradient penalty
                gradient_penalty = calc_gradient_penalty(preD, data.data, 
                                                         fakes.data)
                if iters == 0 and iter_d == 0: plog("gradient_penalty.shape: ",gradient_penalty.shape)
                gradient_penalty.backward()
                GP.append(float(gradient_penalty))
                R_COST.append(float(D_real))
                F_COST.append(float(D_fake))
                Wasserstein_D = D_real - D_fake
                W_DIST.append(float(Wasserstein_D))
                D_cost = D_fake - D_real + gradient_penalty
                #
                optPreD.step()
                D_COST.append(float(D_cost))
            #
            ############################
            # (2) Update G network
            ###########################
            for p in preD.parameters():
                p.requires_grad = False  # to avoid computation
            preG.zero_grad()
            #
            hm = getRandomHM()
            fakes =preG(hm)
            G_cost = preD(fakes)
            G_cost = G_cost.mean().reshape(1)
            G_cost.backward(one)
            optPreG.step()
            G_COST.append(-float(G_cost))
            #
            if iters%5000 == 0 or iters in disp_itrs:
                generate_image([preD, preG, optPreD, optPreG],
                               iters, data.data, 
                               loss_dict1 = {"R_COST":R_COST,
                                            "F_COST":F_COST,
                                            "W_DIST":W_DIST,
                                            #"D_COST":D_COST,
                                            "GP":GP}, 
                               loss_dict2 = {"G_COST":G_COST,
                                            },
                               fakes = fakes.data,
                               imgType = 'hist'
                              )#, fakes.data)
            #if ntbk and iters > 500:
            #    break
        pbar.close()
    torch.save(preG.state_dict(), "preG.mdl")

In [ ]:
n = torch.randn(5)
n.expand( (2, 5) )

In [ ]:
s_noise = torch.randn(opt.nz)
s_style_noise = s_noise.expand((opt.batchSize, opt.nz))

In [ ]:
s_noise = torch.randn(opt.nz)
s_style_noise = s_noise.expand((opt.batchSize, opt.nz))
mse = nn.MSELoss()
if cuda:
    s_style_noise = s_style_noise.cuda(gpu)
    mse = mse.cuda(gpu)
fixed_noise[1][opt.hm_gen_size:] = s_noise

In [ ]:
org_r_state = np.random.get_state()
np.random.seed(2)
#This is being done coz of multi-threading (minimize the effect of drastic change)
same_style_dset = same_style_clocks(randSeed=2, transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Resize(64),
                                transforms.CenterCrop(64),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                            ]))
same_style_dloader = torch.utils.data.DataLoader(
    same_style_dset, batch_size=opt.batchSize,
    shuffle=True, num_workers=int(opt.workers))
np.random.set_state(org_r_state)

In [ ]:
G = GoodGenerator(g_nz)
plog(G)
D = GoodDiscriminator()
plog(D)

In [ ]:
#Now declare the variables!!
optD = optim.Adam(D.parameters(), lr=opt.lrD, betas=(opt.beta1, opt.beta2))
optG = optim.Adam(G.parameters(), lr=opt.lrG, betas=(opt.beta1, opt.beta2))
optGDec = optim.Adam(G.parameters(), lr=opt.lrG, betas=(opt.beta1, opt.beta2))
R_COST = []
F_COST = []
W_DIST = []
D_COST = []
G_COST = []
GP = []

iters = -1
try:
    loadWeights(opt.resumeFldr, loss_dict={
                        "R_COST.pkl": R_COST,
                        "F_COST.pkl": F_COST,
                        "W_DIST.pkl": W_DIST,
                        "D_COST.pkl": D_COST,
                        "G_COST.pkl": G_COST,
                    })
except:
    plog("Encountered error while loading weights!!")

one = torch.FloatTensor([1])
mone = one * -1
if cuda:
    one = one.cuda(gpu)
    mone = mone.cuda(gpu)
    D = D.cuda(gpu)
    G = G.cuda(gpu)

In [ ]:
data_len = len(dataloader)
D_real = 1
D_fake = -1
gradient_penalty = opt.lambda_value
G_cost = 1
DEC_LS = []
loss = 1
for epoch in range(opt.nEpochs):
    plog("Epoch: ", epoch, " of ", opt.nEpochs, " iters: ", iters)
    data_iter = iter(dataloader)
    
    #Now, let's train it
    #This is being doen due to multi-threading
    org_r_state = np.random.get_state()
    np.random.seed(2)
    s_style_iter = iter(same_style_dloader)
    np.random.set_state(org_r_state)
    
    i = 0
    pbar = tqdm(total=len(data_iter)//opt.critic_iters)
    #pbar = tqdm(range(len(data_iter)//opt.critic_iters))
    while i < data_len - opt.critic_iters:
        pbar.set_description( (" D_real {:.3f}, D_fake {:.3f}, "\
        +  "gradient_penalty {:.3f}, G_cost {:.3f}, DEC_loss{:.3f},  {}").format(
            float(D_real), float(D_fake), 
            float(gradient_penalty), float(G_cost), float(loss), iters))
        #pbar.update(1)
        iters += 1
        reals = []
        noises = []
        for iter_d in range(opt.critic_iters):
            reals.append(next(data_iter)[0])
            noises.append(torch.randn(opt.batchSize, g_nz))
        noises.append(torch.randn(opt.batchSize, g_nz))
        do_WPG_iters(reals, noises, 
                     [G, D, optG, optD], 
                     [R_COST, F_COST, GP, W_DIST, D_COST, G_COST])
        #
        ############################################
        # Now one iteration of Decoder training
        ###########################################
        #This is being doen due to multi-threading
        org_r_state = np.random.get_state()
        np.random.seed(2)
        #
        dec_data, H, M = next(s_style_iter)
        np.random.set_state(org_r_state)
        HM = torch.cat([H, M], axis=1).cuda(gpu)
        with torch.no_grad():
            hm_to_g = preG(HM)
            #Add a little noise
            hm_plus_noise = hm_to_g + torch.randn(hm_to_g.shape).cuda(gpu)/1000
            inp_g = torch.cat([hm_plus_noise, s_style_noise], axis = 1)
        #
        optGDec.zero_grad()
        dec_op = G(inp_g)
        loss = mse(dec_data.cuda(gpu), dec_op)
        loss.backward()
        optGDec.step()
        DEC_LS.append(float(loss))
        if iters%5000 == 0 or iters in disp_itrs:
            disp_data = []
            #Fixed time and style  #fixed noise 1 has same style!!! 
            #Clocks 0 adn 1 will have same time as that shown in real
            #Further clock 1 should have same style as well
            for d_i in range(2):
                disp_data.append(dec_data[d_i].cuda(gpu).data)
                fixed_noise[d_i][:opt.hm_gen_size] = hm_to_g[d_i].data
            #These two will be fresh as generated
            for d_i in range(2): disp_data.append(reals[-1][d_i].cuda(gpu).data)
            oToPickle = {}
            oToPickle["fixed_noise"] = fixed_noise
            #preG, optPreG
            oToPickle["optGDec"] = optGDec.state_dict()
            generate_image([D, G, optD, optG],
                           iters, torch.stack(disp_data), 
                           loss_dict1 = {"R_COST":R_COST,
                                        "F_COST":F_COST,
                                        "W_DIST":W_DIST,
                                        #"D_COST":D_COST,
                                        "GP":GP}, 
                           loss_dict2 = {"G_COST":G_COST,
                                        "DEC_LS":DEC_LS,
                                        },
                           oToPickle = oToPickle)#, fakes.data)
        #if ntbk and iters > 500:
        #    break
    pbar.close()
#with batch